
# Project 1: Build an LLM Playground

Welcome! In this project, you’ll learn foundations of large language models (LLMs). We’ll keep the code minimal and the explanations high‑level so that anyone who can run a Python cell can follow along.  

We'll be using Google Colab for this project. Colab is a free, browser-based platform that lets you run Python code and machine learning models without installing anything on your local computer. Click the button below to open this notebook directly in Google Colab and get started!


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mremreozan-bel/ai-eng-projects/blob/main/project_1/lm_playground.ipynb)

---
## Learning Objectives  
* **Tokenization** and how raw text is tokenized into a sequene of discrete tokens
* Inspect **GPT2** and **Transformer architecture**
* Loading pre-trained LLMs using **Hugging Face**
* **Decoding strategies** to generate text from LLMs
* Completion versus **intrusction fine-tuned** LLMs


Let's get started!

In [2]:
import nbformat, os

# Get the current notebook name automatically (works if using Colab local runtime)
from google.colab import _message


In [3]:
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__)
from transformers import AutoTokenizer

torch 2.8.0+cu126 | transformers 4.57.0


# 1 - Tokenization

A neural network can’t digest raw text. They need **numbers**. Tokenization is the process of converting text into IDs. In this section, you'll learn how tokenization is implemented in practice.

Tokenization methods generally fall into three categories:
1. Word-level
2. Character-level
3. Subword-level

### 1.1 - Word‑level tokenization

Split text on whitespace and store each **word** as a token.

In [4]:
# 1. Tiny corpus
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

# 2. Build the vocabulary
PAD, UNK = "[PAD]", "[UNK]"
vocab = []
word2id = {}
id2word = {}

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
all_tokens = set()
for sentence in corpus:
    all_tokens.update(tokenizer.tokenize(sentence))

vocab = [PAD, UNK] + sorted(all_tokens)
word2id = {word: idx for idx, word in enumerate(vocab)}
id2word = {idx: word for idx, word in enumerate(vocab)}

print(f"Vocabulary size: {len(vocab)} words")
print("First 15 vocab entries:", vocab[:15])

# 3. Encode / decode
def encode(text):
    """Convert text to list of token IDs."""
    words = text.lower().split()
    return [word2id.get(word, word2id[UNK]) for word in words]

def decode(ids):
    """Convert list of token IDs back to text."""
    return " ".join([id2word.get(i, UNK) for i in ids])

# 4. Demo
sample = "The brown unicorn jumps"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Vocabulary size: 21 words
First 15 vocab entries: ['[PAD]', '[UNK]', '##ization', 'brown', 'converts', 'dog', 'fox', 'jumps', 'language', 'large', 'lazy', 'models', 'next', 'numbers', 'over']

Input text : The brown unicorn jumps
Token IDs  : [18, 3, 1, 7]
Decoded    : the brown [UNK] jumps


Word-level tokenization has two major limitations:
1. Large vocabulary size
2. Out-of-vocabulary (OOV) issue

### 1.2 - Character‑level tokenization

Every single character (including spaces and emojis) gets its own ID. This guarantees zero out‑of‑vocabulary issues but very long sequences.

In [5]:
# 1. Build a fixed vocabulary # a–z + A–Z + padding + unkwown
import string

vocab = []
char2id = {}
id2char = {}

vocab = ["[PAD]", "[UNK]"] + list(string.ascii_lowercase + string.ascii_uppercase)
char2id = {ch: idx for idx, ch in enumerate(vocab)}
id2char = {idx: ch for idx, ch in enumerate(vocab)}

print(f"Vocabulary size: {len(vocab)} (52 letters + 2 specials)")

# 2. Encode / decode
def encode(text):
    return [char2id.get(ch, char2id["[UNK]"]) for ch in text]

def decode(ids):
    return "".join([id2char.get(i, "[UNK]") for i in ids])

# 3. Demo
sample = "Hello"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)


Vocabulary size: 54 (52 letters + 2 specials)

Input text : Hello
Token IDs  : [35, 6, 13, 13, 16]
Decoded    : Hello


### 1.3 - Subword‑level tokenization

Sub-word methods such as `Byte-Pair Encoding (BPE)`, `WordPiece`, and `SentencePiece` **learn** the most common character and gorup them into new tokens. For example, the word `unbelievable` might turn into three tokens: `["un", "believ", "able"]`. This approach strikes a balance between word-level and character-level methods and fix their limitations.

For example, `BPE` algorithm forms the vocabulary using the following steps:
1. **Start with bytes** → every character is its own token.  
2. **Count all adjacent pairs** in a huge corpus.  
3. **Merge the most frequent pair** into a new token.  
   *Repeat steps 2-3* until you hit the target vocab size (e.g., 50 k).

Let's see `BPE` in practice.

In [6]:
# 1. Load a pretrained BPE tokenizer (GPT-2 uses BPE).
# Refer to  https://huggingface.co/docs/transformers/en/fast_tokenizers

from transformers import AutoTokenizer

bpe_tok = AutoTokenizer.from_pretrained("gpt2")

# 2. Encode / decode
def encode(text):
    return bpe_tok.encode(text, add_special_tokens=False)

def decode(ids):
    return bpe_tok.decode(ids)

# 3. Demo
sample = "Unbelievable tokenization powers! 🚀"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Tokens     :", bpe_tok.convert_ids_to_tokens(ids))
print("Decoded    :", recovered)



Input text : Unbelievable tokenization powers! 🚀
Token IDs  : [3118, 6667, 11203, 540, 11241, 1634, 5635, 0, 12520, 248, 222]
Tokens     : ['Un', 'bel', 'iev', 'able', 'Ġtoken', 'ization', 'Ġpowers', '!', 'ĠðŁ', 'ļ', 'Ģ']
Decoded    : Unbelievable tokenization powers! 🚀


### 1.4 - TikToken

`tiktoken` is a production-ready library which offers high‑speed tokenization used by OpenAI models.  
Let's compare the older **gpt2** encoding with the newer **cl100k_base** used in GPT‑4.

In [7]:
# Use gpt2 and cl100k_base to encode and decode the following text
# Refer to https://github.com/openai/tiktoken
import tiktoken

sentence = "The 🌟 star-player scored 40 points!"

# GPT-2 encoding
gpt2_enc = tiktoken.get_encoding("gpt2")
gpt2_ids = gpt2_enc.encode(sentence)
gpt2_decoded = gpt2_enc.decode(gpt2_ids)
print("GPT-2 IDs   :", gpt2_ids)
print("GPT-2 tokens:", [gpt2_enc.decode([i]) for i in gpt2_ids])
print("GPT-2 decoded:", gpt2_decoded)

# cl100k_base encoding (used by GPT-4)
cl100k_enc = tiktoken.get_encoding("cl100k_base")
cl100k_ids = cl100k_enc.encode(sentence)
cl100k_decoded = cl100k_enc.decode(cl100k_ids)
print("cl100k IDs   :", cl100k_ids)
print("cl100k tokens:", [cl100k_enc.decode([i]) for i in cl100k_ids])
print("cl100k decoded:", cl100k_decoded)

GPT-2 IDs   : [464, 12520, 234, 253, 3491, 12, 7829, 7781, 2319, 2173, 0]
GPT-2 tokens: ['The', ' �', '�', '�', ' star', '-', 'player', ' scored', ' 40', ' points', '!']
GPT-2 decoded: The 🌟 star-player scored 40 points!
cl100k IDs   : [791, 11410, 234, 253, 6917, 43467, 16957, 220, 1272, 3585, 0]
cl100k tokens: ['The', ' �', '�', '�', ' star', '-player', ' scored', ' ', '40', ' points', '!']
cl100k decoded: The 🌟 star-player scored 40 points!


Experiment: try new sentences, emojis, code snippets, or other languages. If you are interested, try implementing the BPE algorithm yourself.

### 1.5 - Key Takeaways

* **Word‑level**: simple but brittle (OOV problems).  
* **Character‑level**: robust but produces long sequences.  
* **BPE / Byte‑Level BPE**: middle ground used by most LLMs.  
* **tiktoken**: shows how production models tokenize with pre‑trained sub‑word vocabularies.

# 2. What is a Language Model?

At its core, a **language model (LM)** is just a *very large* mathematical function built from many neural-network layers.  
Given a sequence of tokens `[t₁, t₂, …, tₙ]`, it learns to output a probability for the next token `tₙ₊₁`.


Each layer applies a simple operation (matrix multiplication, attention, etc.). Stacking hundreds of these layers lets the model capture patterns and statistical relations from text. The final output is a vector of scores that says, “how likely is each possible token to come next?”

> Think of the whole network as **one gigantic equation** whose parameters were tuned during training to minimize prediction error.



### 2.1 - A Single `Linear` Layer

Before we explore Transformer, let’s start tiny:

* A **Linear layer** performs `y = Wx + b`  
  * `x` – input vector  
  * `W` – weight matrix (learned)  
  * `b` – bias vector (learned)

Although this looks basic, chaining thousands of such linear transforms (with nonlinearities in between) gives neural nets their expressive power.


In [8]:
import torch.nn as nn
class Linear(nn.Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x):
        return self.linear(x)

In [9]:
import torch.nn as nn, torch

lin = nn.Linear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))


Input : tensor([ 1.0000, -1.0000,  0.5000])
Weights: Parameter containing:
tensor([[-0.5654,  0.0603,  0.4583],
        [-0.5297,  0.1963,  0.5242]], requires_grad=True)
Bias   : Parameter containing:
tensor([ 0.1465, -0.5532], requires_grad=True)
Output : tensor([-0.2500, -1.0171], grad_fn=<ViewBackward0>)


### 2.2 - A `Transformer` Layer

Most LLMs are a **stack of identical Transformer blocks**. Each block fuses two main components:

| Step | What it does | Where it lives in code |
|------|--------------|------------------------|
| **Multi-Head Self-Attention** | Every token looks at every other token and decides *what matters*. | `block.attn` |
| **Feed-Forward Network (MLP)** | Re-mixes information token-by-token. | `block.mlp` |

Below, we load the smallest public GPT-2 (124 M parameters), grab its *first* block, and inspect the pieces.

Each **Transformer block** in GPT-2 contains the following main components:

| Component | Description |
|------------|--------------|
| **`ln_1`** | Layer Normalization applied **before** the attention mechanism. |
| **`attn`** | **Multi-Head Self-Attention** — allows each token to attend to others in the sequence. |
| **`ln_2`** | Second Layer Normalization applied **before** the feed-forward network. |
| **`mlp`** | **Feed-Forward Network (MLP)** — re-mixes and transforms token representations individually. |


In [10]:
import torch
from transformers import GPT2LMHeadModel

# Load the 124 M-parameter GPT-2 and inspect its layers (12 layers)
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")
# First layer out of 12 layers
block = gpt2.transformer.h[0]
print(block)

GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx=3072)
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)


In [11]:
# Run a tiny forward pass through the first block
seq_len = 8
dummy_tokens = torch.randint(0, gpt2.config.vocab_size, (1, seq_len))
with torch.no_grad():
    # Embed tokens + positions the same way GPT-2 does
    # Forward through one layer
    print(dummy_tokens)
    embeds = gpt2.transformer.wte(dummy_tokens) + gpt2.transformer.wpe(torch.arange(seq_len))
    out = block(embeds)[0]

print("\nOutput shape :", out.shape) # (batch, seq_len, hidden_size)

tensor([[  292, 28630, 39964,  3435, 50132, 23434,  5815, 19624]])

Output shape : torch.Size([1, 8, 768])


### 2.3 - Inside GPT-2

GPT-2 is just many of those modules arranged in a repeating *block*. Let's print the modules inside the Transformer.

In [12]:
# Print the name and modules inside gpt2
for name, module in gpt2.transformer.named_children():
    print(name, ":", module)

wte : Embedding(50257, 768)
wpe : Embedding(1024, 768)
drop : Dropout(p=0.1, inplace=False)
h : ModuleList(
  (0-11): 12 x GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D(nf=2304, nx=768)
      (c_proj): Conv1D(nf=768, nx=768)
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D(nf=3072, nx=768)
      (c_proj): Conv1D(nf=768, nx=3072)
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)
ln_f : LayerNorm((768,), eps=1e-05, elementwise_affine=True)


As you can see, the Transformer holds various modules, arranged from a list of blocks (`h`). The following table summarizes these modules:

| Step | What it does | Why it matters |
|------|--------------|----------------|
| **Token → Embedding** | Converts IDs to vectors | Gives the model a numeric “handle” on words |
| **Positional Encoding** | Adds “where am I?” info | Order matters in language |
| **Multi-Head Self-Attention** | Each token asks “which other tokens should I look at?” | Lets the model relate words across a sentence |
| **Feed-Forward Network** | Two stacked Linear layers with a non-linearity | Mixes information and adds depth |
| **LayerNorm & Residual** | Stabilize training and help gradients flow | Keeps very deep networks trainable |


### 2.4 LLM's output

Passing a token sequence through an **LLM** yields a tensor of **logits** with shape  
`(batch_size, seq_len, vocab_size)`.  
Applying `softmax` on the last dimension turns those logits into probabilities.

The cell below feeds an 8-token dummy sequence, prints the logits shape, and shows the five most likely next tokens for the final position.


In [13]:
import torch, torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Load gpt2 model and tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")

# Tokenize input text
text = "Hello my name"
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]

# Get logits by passing the ids to the gpt2 model.
with torch.no_grad():
    outputs = gpt2(input_ids)
logits = outputs.logits

print("Logits shape :", logits.shape)

# Predict next token
next_token_logits = logits[0, -1, :]
probs = F.softmax(next_token_logits, dim=-1)
topk = torch.topk(probs, 5)

print("\nTop-5 predictions for the next token after 'Hello my name':")
for i, idx in enumerate(topk.indices):
    token = tokenizer.decode(idx)
    prob = topk.values[i].item()
    print(f"{i+1}. {token!r} (p={prob:.4f})")


Logits shape : torch.Size([1, 3, 50257])

Top-5 predictions for the next token after 'Hello my name':
1. ' is' (p=0.7773)
2. ',' (p=0.0373)
3. "'s" (p=0.0332)
4. ' was' (p=0.0127)
5. ' and' (p=0.0076)


### 2.5 - Key Takeaway

A language model is nothing mystical: it’s a *huge composition* of small, understandable layers trained to predict the next token in a sequence of tokens.

# 3 - Generation
Once an LLM is trained to predict the probabilities, we can generate text from the model. This process is called decoding or sampling.

At each step, the LLM outputs a **probability distribution** over the next token. It is the job of the decoding algorithm to pick the next token, and move on to the next token. There are different decoding algorithms and hyper-parameters to control the generaiton:
* **Greedy** → pick the single highest‑probability token each step (safe but repetitive).  
* **Top‑k / Nucleus (top‑p)** → sample from a subset of likely tokens (adds variety).
* **beam** -> applies beam search to pick tokens
* **Temperature** → a *creativity* knob. Higher values flatten the probability distribution.

### 3.1 - Greedy decoding

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
}
tokenizers, models = {}, {}
# Load models and tokenizers
for key, name in MODELS.items():
    tokenizers[key] = AutoTokenizer.from_pretrained(name)
    models[key] = AutoModelForCausalLM.from_pretrained(name)

def generate(tokenizers, models, model_key, prompt, strategy="greedy", max_new_tokens=100, temperature=1.0):
    tok, mdl = tokenizers[model_key], models[model_key]
    # Return the generations based on the provided strategy: greedy, top_k, top_p
    inputs = tok(prompt, return_tensors="pt")

    if strategy == "greedy":
        outputs = mdl.generate(**inputs,
                               max_new_tokens=max_new_tokens,
                               do_sample=False,  # deterministic
                               pad_token_id=tok.eos_token_id)
    elif strategy == "top_k":
        outputs = mdl.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=50,
            temperature=temperature,
            pad_token_id=tok.eos_token_id
        )
    elif strategy == "beam":
        outputs = mdl.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=5,
            early_stopping=True,
            do_sample=False,  # beam search is deterministic
            pad_token_id=tok.eos_token_id
        )
    elif strategy == "top-p":
        outputs = mdl.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=temperature,
            pad_token_id=tok.eos_token_id
        )
    else:
        raise ValueError("Unknown strategy. Use 'greedy', 'top_k', or 'top-p'.")

    return tok.decode(outputs[0], skip_special_tokens=True)

In [17]:
tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Greedy ==")
    print(generate(tokenizers, models, "gpt2", prompt, "greedy", 80))



== GPT-2 | Greedy ==
Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== GPT-2 | Greedy ==
What is 2+2?

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

== GPT-2 | Greedy ==
Suggest a party theme.

The party theme is a simple, simple, and fun way to get your friends to join you.

The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple


Naively picking the single best token every time has the following issues in practice:

* **Loop**: “The cat is is is…”  
* **Miss long-term payoff**: the highest-probability word *now* might paint you into a boring corner later.

### 3.2 - Top-k or top-p sampling

In [20]:
tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Top-p ==")
    print(generate(tokenizers, models, "gpt2", prompt, "top-p", 40))


== GPT-2 | Top-p ==
Once upon a time, in a moment, that we would think it was a sign that the city was at war with the United States, and that the people were coming here to try to control this land, that it

== GPT-2 | Top-p ==
What is 2+2? 2+2, or 2+1?

When you're asking what 2+2 is, it depends on your situation. For example, if the number is less than 1, it might

== GPT-2 | Top-p ==
Suggest a party theme. It helps keep your party safe and has a few good reasons for doing so.

If you need to bring someone with you, you can bring them with you too. Even if you are only


### 3.3 - Try It Yourself

1. Scroll to the list called `tests`.
2. Swap in your own prompts or tweak the decoding strategy.  
3. Re‑run the cell and compare the vibes.

> **Tip:** Try the same prompt with `greedy` vs. `top_p` (0.9) and see how the tone changes. Notice especially how small temperature tweaks can soften or sharpen the prose.

* `strategy`: `"greedy"`, `"beam"`, `"top_k"`, `"top_p"`  
* `temperature`: `0.2 – 2.0`  
* `k` or `p` thresholds



In [28]:
tests = [
    "The secret to happiness is",
    "Artificial intelligence will soon",
    "In the year 2050, humans will",
]

# Common parameters
temperature = 0.8
max_new_tokens = 60

# Strategies to test
strategies = ["greedy", "beam", "top_k", "top-p"]

# Run all strategies for each test prompt
for prompt in tests:
    print("=" * 100)
    print(f"Prompt: {prompt}\n")

    for strategy in strategies:
        print(f"--- Strategy: {strategy} ---")
        try:
            output = generate(tokenizers, models, "gpt2", prompt, strategy=strategy, temperature=temperature, max_new_tokens=max_new_tokens)
            print(output)
        except Exception as e:
            print(f"⚠️ Error with {strategy}: {e}")
        print("\n")

Prompt: The secret to happiness is

--- Strategy: greedy ---
The secret to happiness is to be happy.

The secret to happiness is to be happy.

The secret to happiness is to be happy.

The secret to happiness is to be happy.

The secret to happiness is to be happy.

The secret to happiness is to be happy.



--- Strategy: beam ---
The secret to happiness is that you don't have to be happy to be happy. You don't have to be happy to be happy. You don't have to be happy to be happy. You don't have to be happy to be happy. You don't have to be happy to be happy. You don't have


--- Strategy: top_k ---
The secret to happiness is to be always optimistic. It means knowing that your life is going to be the best it ever is, and you're going to be able to live with all these wonderful things that are going to come about.

The only way to know when and how to feel good about yourself is to become


--- Strategy: top-p ---
The secret to happiness is to have a good time with others, but they are al

# 4 - Completion vs. Instruction-tuned LLMs

We have seen that we can use GPT2 model to pass an input text and generate a new text. However, this model only continues the provided text. It is not engaging in a dialouge-like conversation and cannot be helpful by answering instructions. On the other hand, **instruction-tuned LLMs** like `Qwen-Chat` go through an extra training stage called **post-training** after the base “completion” model is finished. Because of post-training step, an instruction-tuned LLM will:

* **Read the entire prompt as a request,** not just as text to mimic.  
* **Stay in dialogue mode**. Answer questions, follow steps, ask clarifying queries.  
* **Refuse or safe-complete** when instructions are unsafe or disallowed.  
* **Adopt a consistent persona** (e.g., “Assistant”) rather than drifting into story continuation.


### 4.1 - Qwen1.5-8B vs. GPT2

In the code below we’ll feed the same prompt to:

* **GPT-2 (completion-only)** – it will simply keep writing in the same style.  
* **Qwen-Chat (instruction-tuned)** – it will obey the instruction and respond directly.

Comparing the two outputs makes the difference easy to see.

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
    "qwen": "Qwen/Qwen1.5-1.8B-Chat"
}
tokenizers, models = {}, {}
# Load models and tokenizers
for key, name in MODELS.items():
    tokenizers[key] = AutoTokenizer.from_pretrained(name)
    models[key] = AutoModelForCausalLM.from_pretrained(name)



We downloaded two tiny checkpoints: `GPT‑2` (124 M parameters) and `Qwen‑1.5‑Chat` (1.8 B). If the cell took a while, that was mostly network time. Models are stored locally after the first run.

Let's now generate text and compare two models.


In [9]:
tests=[("Once upon a time","greedy"),("What is 2+2?","top_k"),("Suggest a party theme.","top-p")]
for prompt,strategy in tests:
    print("=" * 100)
    print(f"Prompt: {prompt}\n")
    for key in ["gpt2","qwen"]:
        print(f"\n== {key.upper()} | {strategy} ==")
        print(generate(tokenizers, models, key,prompt,strategy, max_new_tokens=80))

Prompt: Once upon a time


== GPT2 | greedy ==


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== QWEN | greedy ==
Once upon a time, there was a young girl named Lily who lived in a small village nestled among the rolling hills of her home. Lily had always been fascinated by nature and spent most of her days exploring the surrounding forests, streams, and meadows. She loved to sing songs about the birds that sang in the trees, the flowers that bloomed in the fields, and the animals that roamed free.
One day
Prompt: What is 2+2?


== GPT2 | top_k ==
What is 2+2?

In this blog post, we're going to be discussing some interesting scenarios. It will also be more than a little humorous, but it will provide some interesting back

# 5. (Optional) A Small LLM Playground

### 5.1 ‑ Interactive Playground

Enter a prompt, pick a model and decoding strategy, adjust the temperature, and press **Generate** to watch the model respond.


In [10]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Make sure models and tokenizers are loaded
try:
    tokenizers
    models
except NameError:
    raise RuntimeError("Please run the earlier setup cells that load the models before using the playground.")

# Dropdown for model selection
model_dropdown = widgets.Dropdown(
    options=list(MODELS.keys()),
    value="gpt2",
    description="Model:",
    style={'description_width': 'initial'},
)

# Dropdown for decoding strategy
strategy_dropdown = widgets.Dropdown(
    options=["greedy", "beam", "top_k", "top-p"],
    value="top-p",
    description="Strategy:",
    style={'description_width': 'initial'},
)

# Temperature slider
temperature_slider = widgets.FloatSlider(
    value=0.8,
    min=0.2,
    max=2.0,
    step=0.1,
    description="Temperature:",
    continuous_update=False,
    readout_format=".1f",
    style={'description_width': 'initial'},
)

# Text area for prompt input
prompt_box = widgets.Textarea(
    value="Write a short poem about the ocean:",
    placeholder="Type your prompt here...",
    description="Prompt:",
    layout=widgets.Layout(width="100%", height="120px"),
    style={'description_width': 'initial'},
)

# Button to trigger generation
generate_button = widgets.Button(
    description="Generate",
    button_style="success",
    tooltip="Click to generate text",
    icon="bolt",
)

# Output area for the model response
output_area = widgets.Output()

# Function to handle button click
def on_generate_clicked(b):
    with output_area:
        output_area.clear_output()
        model_key = model_dropdown.value
        strategy = strategy_dropdown.value
        temperature = temperature_slider.value
        prompt = prompt_box.value
        print("=" * 100)
        print(f"Prompt: {prompt}\n")
        print(f"🧠 Model: {model_key} | Strategy: {strategy} | Temperature: {temperature}\n")
        result = generate(tokenizers, models, model_key, prompt, strategy=strategy, temperature=temperature, max_new_tokens=100)
        display(Markdown(f"**Output:**\n\n{result}"))

# Connect the button to the function
generate_button.on_click(on_generate_clicked)

# Display the full UI
ui = widgets.VBox([
    prompt_box,
    widgets.HBox([model_dropdown, strategy_dropdown]),
    temperature_slider,
    generate_button,
    output_area
])

display(ui)



## 🎉 Congratulations!

You’ve just learned, explored, and inspected a real **LLM**. In one project you:
* Learned how **tokenization** works in practice
* Used `tiktoken` library to load and experiment with most advanced tokenizers.
* Explored LLM architecture and inspected GPT2 blocks and layers
* Learned decoding strategies and used `top-p` to generate text from GPT2
* Loaded a powerful chat model, `Qwen1.5-8B` and generated text
* Built an LLM playground


👏 **Great job!** Take a moment to celebrate. You now have a working mental model of how LLMs work. The skills you used here power most LLMs you see everywhere.
